In [8]:
from bs4 import BeautifulSoup
import requests 
import lxml
import cnfg
import tweepy
import json
import selenium
from selenium import webdriver
import time
import numpy as np
import pandas as pd
from itertools import cycle
from tweepy.error import TweepError
import os

## Let's play with the emojitracker website!  

This is **EASILY** the dumbest thing I've coded to date, but it's *mildely* entertaining.  It flashes patterns accross [emojitracker.com](http://emojitracker.com) by first scraping the current rankings grid, and then tweeting the relevant emojies in patterns accross a pre-defined matrix.  

Additionally it saves the emoji rankings in a json file for future highly technichal and monumentally earthshattering analysis.  Undoubtedly the keys to predicting the stockmarket **AND** global warning are contained within the little yellow replacements for human contact and their relative frequency on twitter. Time to nerd out boys and girls! 

To set up/run:  
1. Put your relevant api info in the correct params in a call to setup() and then run the notebook.  
2. Give [Mroth](https://github.com/mroth) a high five if you know him because his code is dope!

In [ ]:
# run once, if you run this more than 
# that it means you've opend the notebook 
# more than once and you really shoud spend 
# your time more wisely. Have you not heard of videogames?
# Your mother would be furious

def setup(consumer_key,consumer_secret ,access_token, access_token_secret):
    d = dict(consumer_key=consumerKey, consumer_secret=consumer_secret,
             access_token=access_token, access_token_secret=access_token_secret)
    with open(os.path.expanduser('~/.twitmoji_app')) as configurator5000:
        json.dump(d,configurator5000)
    print("Amazing work, take a break you've earned it")

In [282]:
def initAPI():
    config = cnfg.load(".twitmoji_app")
    key = config["consumer_key"]
    cSecret = config["consumer_secret"]
    token = config["access_token"]
    tSecret = config["access_token_secret"]
    auth = tweepy.OAuthHandler(consumer_key=key,consumer_secret=cSecret)
    auth.set_access_token(token, tSecret)
    tweet = tweepy.API(auth_handler=auth)
    return tweet


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def getmojigrid():
    '''
    uses selenium to get emojidata from emojitracker.com
    saves a timestamped json file of emoji rankings in 
    reporoot/data for important future analysis
    '''
    url = 'http://www.emojitracker.com/'
    browser = webdriver.Firefox()
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    browser.close()
    data = [i.text.strip("'").strip() for i in soup.find(id='data').findAll('a')]
    ranks = []
    ranksd = {}
    hist={}
    for elem in data:
        l = list(elem)
        emoji = l[0]
        rank = int(''.join(c for c in l[1:] if c.isdigit()))
        ranks.append((emoji,rank))
        ranksd[emoji] = rank
    emojies = [i[0] for i in ranks]
    counts = [i[1] for i in ranks]
    e_matrix = [i for i in chunks(emojies,8)]
    c_matrix = [i for i in chunks(counts,8)]
    e_grid = pd.DataFrame(e_matrix)
    c_grid = pd.DataFrame(c_matrix)
    now = str(round(time.time()))
    with open('data/emojidata.json', 'r') as readjson:
        hist = json.load(readjson)
    hist['{}'.format(now)] = ranksd
    with open('data/emojidata.json','w') as writejson:
        json.dump(hist,writejson)
    return dict(grid=e_grid, freq=c_grid, shape=e_grid.shape, ranks=ranks, counts=counts, hist=hist )


def square_window(frame, start_x, start_y, width):
    '''
    frame = data['grid'] from data = getmojigrid()
    start_x = x coord for top left corner
    start_y = y coord for top left corner
    width = number of cells wide max = 8 
    returns a roughly square matrix as represented on emojitracker
    
    Things work better towards the bottom of the matrix due to the epilepsy happening up top.
    70 or so is a good start_x value
    '''
    if  width - start_y > 8:
        width = 8-start_y
    return frame.iloc[start_x:round(start_x+(width*4.2)),start_y:start_y+width]  


def get_lim(twitter):
    '''
    makes twitter limit rates human readable
    pass tweepy api object
    '''
    this = twitter.rate_limit_status()
    d = {}
    for key in this['resources']:
        for ep in this['resources']['{}'.format(key)]:
            r =this['resources']['{}'.format(key)]['{}'.format(ep)]['remaining']
            l = this['resources']['{}'.format(key)]['{}'.format(ep)]['limit']
            d['{}'.format(ep)] = l - r
    for key in d:
        if d["{}".format(key)] != 0:
            print(str(key)+ ' ' + str(d["{}".format(key)]))

### The patterns so far

In [225]:

def snow(twitterAPI,square,n_tweets,delay):
    '''
    random flashing on a pre-defined square
    twitterAPI = api object
    square = grid of emoji coords from square_window()
    delay = how long to wait so you're not an ass
    returns n_tweets * delay seconds of enjoyment
    '''
    counter = 0
    dim_x = square.shape[0]
    dim_y = square.shape[1]
    while counter <= n_tweets:
        x = np.random.choice(dim_x)
        y = np.random.choice(dim_y)
        target = square.iloc[x,y]
#         salting the tweets so status update isn't blocked due to repetition
        salt = round(np.random.randn(),10)
        s_string = target +"   "+str(salt)
        status = twitterAPI.update_status(s_string)
        counter += 1
        time.sleep(delay)
#         get rid of em
        status.destroy()
    return        


def barber_pole(twitterAPI, square,delay):
    '''
    flash down diagonally on a pre-defined square
    twitterAPI = api object
    square = grid of emoji coords from square_window()
    delay = how long to wait so you're not an ass
    returns (square.shape[1] * delay) seconds of enjoyment
    '''
    dim_x = square.shape[0]
    dim_y = square.shape[1]
    down = range(dim_y)
    right = range(dim_x)
    tupgen = zip(cycle(right),cycle(down))
    pairs = []
    while len(pairs)<=dim_x:
        pairs.append(tupgen.__next__())
    x = [i[0] for i in pairs]
    y = [i[1] for i in pairs]
    for p in pairs:
        target = square.iloc[p[0],p[1]]
        salt = round(np.random.randn(),10)
        s_string = target +"   "+str(salt)
        status = twitterAPI.update_status(s_string)
        time.sleep(delay)
        status.destroy()
            

In [289]:
# this is ephemeral, it happens in real time.  have emojitracker.com 
# open and scrolled to near the bottom or it will do litterally nothing for you

def give_it_a_try():
    tweet = initAPI()
    data = getmojigrid()
    sw = square_window(data['grid'], 71,0,7)
    barber_pole(sw)
    return tweet, data
    

In [ ]:
tweet,data = give_it_a_try()